# IBM Data Science Specialization Capstone Project Notebook

In [80]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Part 1
### Get HTML data from wikipedia

In [2]:
Canada_M = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Scrape PostalCode, Borough, Neighborhood from HTML using BeautifulSoup

In [3]:
# soup = BeautifulSoup(open("Canada_M.htm"), "lxml")
soup = BeautifulSoup(Canada_M, "lxml")

In [4]:
PostalCode = []
Borough = []
Neighborhood = []

for data in soup.tbody.find_all('tr')[1:]:
    PostalCode.append(data.find('td').text)
    Borough.append(data.find_all('td')[1].text)
    Neighborhood.append(data.find_all('td')[2].text[:-1])

In [5]:
data = {'PostalCode': PostalCode,
        'Borough': Borough,
        'Neighborhood': Neighborhood}
df = pd.DataFrame(data)
# Drop rows where Borough is 'Not assigned', then reset index
df = df[df.Borough != 'Not assigned'].reset_index().drop('index', axis = 1)
df.head()

PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

### Combine Neighborhood with same PostalCode

In [6]:
i = 1
while(i < len(df)):
    if df['PostalCode'].iloc[i] == df['PostalCode'].iloc[i - 1]:
        df.at[i - 1, 'Neighborhood'] = df.Neighborhood.iloc[i - 1] +', ' + df.Neighborhood.iloc[i]
        df.drop(index = i, inplace = True)
        df = df.reset_index().drop('index', axis = 1)
    else:
        i += 1

In [7]:
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Harbourfront, Regent Park
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A      Queen's Park                      Not assigned

### Get the number of rows in the datafrome which is 103

In [8]:
df.shape

(103, 3)

## Part 2

### Achieve Latitude and Longitude of each PostalCode

In [9]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format('PGQ7+MV'))
#     lat_lng_coords = g.latlng
#     print('1')

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


In [10]:
# Rename the 'Postal Code' column
Geo = pd.read_csv("Geospatial_Coordinates.csv") 
Geo.rename({'Postal Code': 'PostalCode'}, axis='columns', inplace=True)

In [11]:
# Merge Latitude and longitude to original dataframe
df = pd.merge(df, Geo, on='PostalCode')
df

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A      Queen's Park   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
..         ...               ...   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
95         M1X       Scarborough   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Harbourfront, Regent Park  43.654260 -79.360636  
3                     Lawrence Heights, Lawrence Manor  43.718518 -79.464763  
4                                         Not assigned  43.662301 -79.389494  
5                                     Islington Avenue  43.667856 -79.532242  
6                                       Rouge, Malvern  43.806686 -79.194353  
7                                      Don Mills North  43.745906 -79.352188  
8                      Woodbine Gardens, Parkview Hill  43.706397 -79.309937  
9                             Ryerson, Garden District  43.657162 -79.378937  
10                                           Glencairn  43.709577 -79.445073  
11   Cloverdale, Islington, Martin Grove, Princess ...  43.650943 -79.554724  
12              Highland Creek, Rouge Hill, Port Union  43.784535 -79.160497  
13                    Flemingdon Park, Don Mills South  43.725900 -79.340923  
14                                    Woodbine Heights  43.695344 -79.318389  
15                                      St. James Town  43.651494 -79.375418  
16                                  Humewood-Cedarvale  43.693781 -79.428191  
17   Bloordale Gardens, Eringate, Markland Wood, Ol...  43.643515 -79.577201  
18                   Guildwood, Morningside, West Hill  43.763573 -79.188711  
19                                         The Beaches  43.676357 -79.293031  
20                                         Berczy Park  43.644771 -79.373306  
21                               

## Part 3

### Create a map of Toronto with neighborhoods superimposed on top.

In [31]:
import folium
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[43.706204, -79.398752], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#e182f2',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
# Show map Toronto with borough and neighborhood 
map_Toronto

#### Define Foursquare Credentials and Version

In [29]:
CLIENT_ID = 'PESQIFIZTNDBTMZPSPLZXDCQVGVQT50IE3K1RP3QCQALSDCV' # your Foursquare ID
CLIENT_SECRET = 'ODUYPM1K0ZG1Z0LZTM4GE5IRA0KSMBARQMVDJWAD0HC04UBW' # your Foursquare Secret
VERSION = '20181018' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PESQIFIZTNDBTMZPSPLZXDCQVGVQT50IE3K1RP3QCQALSDCV
CLIENT_SECRET:ODUYPM1K0ZG1Z0LZTM4GE5IRA0KSMBARQMVDJWAD0HC04UBW


In [116]:
df[df.Neighborhood == 'Upper Rouge']

PostalCode      Borough Neighborhood   Latitude  Longitude
95        M1X  Scarborough  Upper Rouge  43.836125 -79.205636

In [117]:
neighborhood_latitude = df.loc[95, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[95, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[95, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Upper Rouge are 43.836124700000006, -79.20563609999999.


### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

In [126]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000# define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PESQIFIZTNDBTMZPSPLZXDCQVGVQT50IE3K1RP3QCQALSDCV&client_secret=ODUYPM1K0ZG1Z0LZTM4GE5IRA0KSMBARQMVDJWAD0HC04UBW&v=20181018&ll=43.836124700000006,-79.20563609999999&radius=2000&limit=100'

In [127]:
results = requests.get(url).json()

In [128]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [129]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

name        categories        lat        lng
0    Pumpkinland (Whittamore Farms)              Farm  43.849236 -79.206647
1                 Clownfish Village  Sculpture Garden  43.821000 -79.205396
2                   Taj supermarket     Grocery Store  43.818746 -79.210172
3  Cedar Brae Golf and Country Club       Golf Course  43.839154 -79.230089

And how many venues were returned by Foursquare?

In [66]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("Finished!")
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [130]:
# type your answer here

Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                 latitudes=df['Latitude'],
                                 longitudes=df['Longitude'],
                                 radius = 2000
                                )



Finished!


#### Let's check the size of the resulting dataframe

In [131]:
print(Toronto_venues.shape)
Toronto_venues.head()

(8511, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Parkwoods              43.753259              -79.329656   
1    Parkwoods              43.753259              -79.329656   
2    Parkwoods              43.753259              -79.329656   
3    Parkwoods              43.753259              -79.329656   
4    Parkwoods              43.753259              -79.329656   

                         Venue  Venue Latitude  Venue Longitude  \
0              Allwyn's Bakery       43.759840       -79.324719   
1  Donalda Golf & Country Club       43.752816       -79.342741   
2           Darband Restaurant       43.755194       -79.348498   
3              Brookbanks Park       43.751976       -79.332140   
4                   LA Fitness       43.747665       -79.347077   

              Venue Category  
0       Caribbean Restaurant  
1                Golf Course  
2  Middle Eastern Restaurant  
3                       Park  
4       Gym / Fitness Center

In [132]:
Toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Agincourt                                                             100   
Agincourt North, L'Amoreaux East, Milliken, Ste...                    100   
Albion Gardens, Beaumond Heights, Humbergate, J...                     46   
Alderwood, Long Branch                                                100   
Bathurst Manor, Downsview North, Wilson Heights                        55   
Bayview Village                                                        47   
Bedford Park, Lawrence Manor East                                     100   
Berczy Park                                                           100   
Birch Cliff, Cliffside West                                            44   
Bloordale Gardens, Eringate, Markland Wood, Old...                     29   
Brockton, Exhibition Place, Parkdale Village                          100   
Business reply mail Processing Centre969 Eastern                      100   
CFB Toronto, Downsview East                                           100   
CN Tower, Bathurst Quay, Island airport, Harbou...                    100   
Cabbagetown, St. James Town                                           100   
Caledonia-Fairbanks                                                   100   
Canada Post Gateway Processing Centre                                 100   
Cedarbrae                                                             100   
Central Bay Street                                                    100   
Chinatown, Grange Park, Kensington Market                             100   
Christie                                                              100   
Church and Wellesley                                                  100   
Clairlea, Golden Mile, Oakridge                                        92   
Clarks Corners, Sullivan, Tam O'Shanter                                79   
Cliffcrest, Cliffside, Scarborough Village West                        37   
Cloverdale, Islington, Martin Grove, Princess G...                     49   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                            100   
Davisville North                                                      100   
...                                                                   ...   
Not assigned                                                          100   
Parkdale, Roncesvalles                                                100   
Parkwoods                                                              95   
Rosedale                                                              100   
Roselawn                                                              100   
Rouge, Malvern                                                         46   
Runnymede, Swansea                                                    100   
Ryerson, Garden District                                              100   
Scarborough Village                                                    72   
Silver Hills, York Mills                                              100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                       100   
Studio District                                                       100   
The Annex, North Midtown, Yorkville                                   100   
The Beaches                                                           100   
The Beaches West, India Bazaar                                        100   
The Danforth West, Riverdale                                          100   
The Junction North, Runnymede                                          77   
The Kingsway, Montgomery Road, Old Mill North                         100   
Thorncliffe Park                                

#### Let's find out how many unique categories can be curated from all the returned venues

In [133]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 323 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [134]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Zoo Exhibit  Accessories Store  Adult Boutique  Afghan Restaurant  \
0            0                  0               0                  0   
1            0                  0               0                  0   
2            0                  0               0                  0   
3            0                  0               0                  0   
4            0                  0               0                  0   

   African Restaurant  Airport  Airport Lounge  American Restaurant  \
0                   0        0               0                    0   
1                   0        0               0                    0   
2                   0        0               0                    0   
3                   0        0               0                    0   
4                   0        0               0                    0   

   Amphitheater  Animal Shelter ...   Vietnamese Restaurant  Volleyball Court  \
0             0               0 ...                       0                 0   
1             0               0 ...                       0                 0   
2             0               0 ...                       0                 0   
3             0               0 ...                       0                 0   
4             0               0 ...                       0                 0   

   Warehouse Store  Whisky Bar  Wine Bar  Wine Shop  Wings Joint  \
0                0           0         0          0            0   
1                0           0         0          0            0   
2                0           0         0          0            0   
3                0           0         0          0            0   
4                0           0         0          0            0   

   Women's Store  Yoga Studio  Zoo  
0              0            0    0  
1              0            0    0  
2              0            0    0  
3              0            0    0  
4              0            0    0  

[5 rows x 323 columns]

And let's examine the new dataframe size.

In [135]:
Toronto_onehot.shape

(8511, 323)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [136]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

Neighborhood  Zoo Exhibit  \
0                           Adelaide, King, Richmond          0.0   
1                                          Agincourt          0.0   
2  Agincourt North, L'Amoreaux East, Milliken, St...          0.0   
3  Albion Gardens, Beaumond Heights, Humbergate, ...          0.0   
4                             Alderwood, Long Branch          0.0   

   Accessories Store  Adult Boutique  Afghan Restaurant  African Restaurant  \
0                0.0             0.0                0.0                 0.0   
1                0.0             0.0                0.0                 0.0   
2                0.0             0.0                0.0                 0.0   
3                0.0             0.0                0.0                 0.0   
4                0.0             0.0                0.0                 0.0   

   Airport  Airport Lounge  American Restaurant  Amphitheater ...   \
0      0.0             0.0                 0.01           0.0 ...    
1      0.0             0.0                 0.02           0.0 ...    
2      0.0             0.0                 0.00           0.0 ...    
3      0.0             0.0                 0.00           0.0 ...    
4      0.0             0.0                 0.01           0.0 ...    

   Vietnamese Restaurant  Volleyball Court  Warehouse Store  Whisky Bar  \
0                   0.00               0.0              0.0         0.0   
1                   0.00               0.0              0.0         0.0   
2                   0.03               0.0              0.0         0.0   
3                   0.00               0.0              0.0         0.0   
4                   0.00               0.0              0.0         0.0   

   Wine Bar  Wine Shop  Wings Joint  Women's Store  Yoga Studio  Zoo  
0       0.0        0.0         0.00            0.0         0.01  0.0  
1       0.0        0.0         0.01            0.0         0.00  0.0  
2       0.0        0.0         0.01            0.0         0.00  0.0  
3       0.0        0.0         0.00            0.0         0.00  0.0  
4       0.0        0.0         0.02            0.0         0.00  0.0  

[5 rows x 323 columns]

#### Let's confirm the new size

In [137]:
Toronto_grouped.shape

(103, 323)

#### Let's print each neighborhood along with the top 5 most common venues

In [138]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.06
1        Hotel  0.04
2         Café  0.04
3      Theater  0.04
4    Gastropub  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.12
1         Coffee Shop  0.07
2            Pharmacy  0.04
3              Bakery  0.04
4   Indian Restaurant  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.19
1         Coffee Shop  0.07
2     Bubble Tea Shop  0.04
3              Bakery  0.04
4                Park  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.17
1  Fast Food Restaurant  0.11
2     Indian Restaurant  0.09
3           Pizza Place  0.09
4         Grocery Store  0.07


----Alderwood, Long Branch----
                    venue  freq
0             Coffee Shop  0.11
1    Fast Food Restau

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [139]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                          Agincourt    Chinese Restaurant   
2  Agincourt North, L'Amoreaux East, Milliken, St...    Chinese Restaurant   
3  Albion Gardens, Beaumond Heights, Humbergate, ...           Coffee Shop   
4                             Alderwood, Long Branch           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                 Hotel                  Café               Theater   
1           Coffee Shop                Bakery              Pharmacy   
2           Coffee Shop                  Park           Pizza Place   
3  Fast Food Restaurant           Pizza Place     Indian Restaurant   
4           Pizza Place  Fast Food Restaurant      Department Store   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Italian Restaurant           Pizza Place          Concert Hall   
1   Japanese Restaurant     Indian Restaurant            Restaurant   
2       Bubble Tea Shop                Bakery  Cantonese Restaurant   
3         Grocery Store                  Park    Italian Restaurant   
4    Seafood Restaurant                  Café              Pharmacy   

   8th Most Common Venue 9th Most Common Venue  10th Most Common Venue  
0        Thai Restaurant             Gastropub           Burrito Place  
1         Sandwich Place  Cantonese Restaurant    Caribbean Restaurant  
2  Vietnamese Restaurant          Dessert Shop               BBQ Joint  
3    Fried Chicken Joint            Beer Store          Sandwich Place  
4         Clothing Store        Sandwich Place  Furniture / Home Store

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [145]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 4, 4, 1, 4, 4, 1, 1, 1, 0, 4, 4,
       0, 1, 1, 1, 1, 0, 4, 4, 4, 1, 0, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 0,
       1, 4, 1, 1, 4, 0, 4, 4, 1, 1, 4, 4, 1, 4, 1, 1, 1, 1, 1, 4, 1, 4,
       1, 1, 4, 1, 4, 1, 1, 0, 4, 1, 4, 4, 3, 4, 0, 1, 1, 0, 0, 4, 4, 4,
       4, 4, 4, 4, 1, 2, 1, 1, 1, 0, 1, 1, 1, 4, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [146]:
Toronto_merged = df

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Harbourfront, Regent Park  43.654260   
3        M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4        M7A      Queen's Park                      Not assigned  43.662301   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.329656               0           Coffee Shop   Japanese Restaurant   
1 -79.315572               1           Coffee Shop  Fast Food Restaurant   
2 -79.360636               1           Coffee Shop                  Café   
3 -79.464763               1        Clothing Store           Coffee Shop   
4 -79.389494               1                  Café           Coffee Shop   

   3rd Most Common Venue 4th Most Common Venue   5th Most Common Venue  \
0               Pharmacy           Pizza Place      Chinese Restaurant   
1                    Gym         Grocery Store          Clothing Store   
2                   Park    Italian Restaurant              Restaurant   
3  Vietnamese Restaurant  Fast Food Restaurant  Furniture / Home Store   
4    Japanese Restaurant                  Park      Mexican Restaurant   

           6th Most Common Venue 7th Most Common Venue  \
0                    Supermarket          Intersection   
1                 Sandwich Place   Japanese Restaurant   
2                            Pub             Gastropub   
3                  Grocery Store        Cosmetics Shop   
4  Vegetarian / Vegan Restaurant           Pizza Place   

       8th Most Common Venue 9th Most Common Venue    10th Most Common Venue  
0             Sandwich Place                   Gym                      Café  
1  Middle Eastern Restaurant            Beer Store  Mediterranean Restaurant  
2                     Bakery   Japanese Restaurant            Farmers Market  
3        Fried Chicken Joint          Liquor Store            Sandwich Place  
4               Concert Hall             Bookstore          Sushi Restaurant

Finally, let's visualize the resulting clusters

In [149]:
# create map
map_clusters = folium.Map(location=[43.706204, -79.398752], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [150]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0         North York               0           Coffee Shop   
8          East York               0           Pizza Place   
19      East Toronto               0           Coffee Shop   
22       Scarborough               0           Coffee Shop   
27        North York               0           Coffee Shop   
32       Scarborough               0  Fast Food Restaurant   
43      West Toronto               0                  Café   
49        North York               0           Coffee Shop   
73   Central Toronto               0           Coffee Shop   
80  Downtown Toronto               0                  Café   
83   Central Toronto               0                  Park   
84  Downtown Toronto               0                  Café   
97  Downtown Toronto               0                  Café   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0     Japanese Restaurant              Pharmacy           Pizza Place   
8             Coffee Shop                  Park        Sandwich Place   
19                    Pub                  Café                   Bar   
22   Fast Food Restaurant              Pharmacy        Sandwich Place   
27    Japanese Restaurant                  Bank    Chinese Restaurant   
32            Coffee Shop           Pizza Place        Sandwich Place   
43            Coffee Shop            Restaurant                   Bar   
49  Vietnamese Restaurant  Fast Food Restaurant           Pizza Place   
73     Italian Restaurant                  Café      Sushi Restaurant   
80            Coffee Shop                  Park                   Bar   
83     Italian Restaurant      Sushi Restaurant                  Café   
84            Coffee Shop    Italian Restaurant                   Bar   
97                  Hotel               Theater           Coffee Shop   

     5th Most Common Venue          6th Most Common Venue  \
0       Chinese Restaurant                    Supermarket   
8                 Pharmacy                     Beer Store   
19                  Bakery                 Breakfast Spot   
22  Furniture / Home Store                    Supermarket   
27          Sandwich Place                    Pizza Place   
32          Discount Store                       Pharmacy   
43        Asian Restaurant  Vegetarian / Vegan Restaurant   
49  Furniture / Home Store                 Sandwich Place   
73                  Bakery           Fast Food Restaurant   
80          Sandwich Place                    Pizza Place   
83             Coffee Shop                         Bakery   
84          Sandwich Place                            Gym   
97      Italian Restaurant                   Concert Hall   

            7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0                    Intersection        Sandwich Place                   Gym   
8                          Bakery    Athletics & Sports            Restaurant   
19                          Beach        Ice Cream Shop       Thai Restaurant   
22             Chinese Restaurant        Discount Store           Pizza Place   
27               Sushi Restaurant              Pharmacy         Grocery Store   
32                  Grocery Store    Chinese Restaurant            Beer Store   
43                         Bakery    Mexican Restaurant     French Restaurant   
49                    Supermarket                  Bank         Grocery Store   
73                   Burger Joint   Japanese Restaurant     Indian Restaurant   
80  Vegetarian / Vegan Restaurant    Italian Restaurant    Mexican Restaurant   
83              Indian Restaurant          Dessert Shop           Wings Joint   
84                       Beer Bar           Yoga Studio           Pizza Place   
97                      Gastropub           Pizza Place              Beer Bar   

           10th Most Common Venue  
0                            Café  
8                   Grocery Store  
19                 Sand

#### Cluster 2

In [151]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels  1st Most Common Venue  \
1          North York               1            Coffee Shop   
2    Downtown Toronto               1            Coffee Shop   
3          North York               1         Clothing Store   
4        Queen's Park               1                   Café   
5           Etobicoke               1          Grocery Store   
6         Scarborough               1            Zoo Exhibit   
7          North York               1            Coffee Shop   
9    Downtown Toronto               1              Gastropub   
10         North York               1         Clothing Store   
13         North York               1            Coffee Shop   
16               York               1     Italian Restaurant   
17          Etobicoke               1                   Bank   
18        Scarborough               1            Pizza Place   
23          East York               1            Coffee Shop   
24   Downtown Toronto               1                   Café   
25   Downtown Toronto               1                   Café   
26        Scarborough               1            Coffee Shop   
31       West Toronto               1                   Café   
33         North York               1            Coffee Shop   
34         North York               1            Coffee Shop   
36   Downtown Toronto               1                   Café   
37       West Toronto               1                   Café   
38        Scarborough               1   Fast Food Restaurant   
39         North York               1     Chinese Restaurant   
41       East Toronto               1       Greek Restaurant   
42   Downtown Toronto               1                   Café   
44        Scarborough               1            Coffee Shop   
46         North York               1  Vietnamese Restaurant   
47       East Toronto               1                   Café   
52         North York               1      Korean Restaurant   
53         North York               1  Vietnamese Restaurant   
56               York               1            Coffee Shop   
58        Scarborough               1            Coffee Shop   
59         North York               1      Korean Restaurant   
60         North York               1            Coffee Shop   
61    Central Toronto               1            Coffee Shop   
62    Central Toronto               1            Coffee Shop   
64               York               1            Coffee Shop   
66         North York               1            Coffee Shop   
67    Central Toronto               1            Coffee Shop   
69       West Toronto               1                   Café   
71        Scarborough               1            Coffee Shop   
72         North York               1            Coffee Shop   
75       West Toronto               1            Coffee Shop   
81       West Toronto               1                   Park   
82        Scarborough               1   Fast Food Restaurant   
92   Downtown Toronto               1     Italian Restaurant   
94          Etobicoke               1            Coffee Shop   
95        Scarborough               1            Golf Course   
96   Downtown Toronto               1            Coffee Shop   
98          Etobicoke               1            Coffee Shop   
99   Downtown Toronto               1                   Park   
100      East Toronto               1            Coffee Shop   
102         Etobicoke               1            Coffee Shop   

    2nd Most Common Venue      3rd Most Common Venue  \
1    Fast Food Restaurant                        Gym   
2                    Café                       Park   
3             Coffee Shop      Vietnamese Restaurant   
4             Coffee Shop        Japanese Restaurant   
5                Pharmacy                Coffee Shop   
6    Fast Food Restaurant                Pizza Place   
7     Japanese Restaurant                       Park   
9                    Café                Coff

#### Cluster 3

In [152]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
93  Etobicoke               2           Coffee Shop           Pizza Place   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
93  Fast Food Restaurant      Department Store    Seafood Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
93                  Café              Pharmacy        Clothing Store   

   9th Most Common Venue  10th Most Common Venue  
93        Sandwich Place  Furniture / Home Store

#### Cluster 4

In [153]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
78  Scarborough               3    Chinese Restaurant           Coffee Shop   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
78                Bakery              Pharmacy   Japanese Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
78     Indian Restaurant            Restaurant        Sandwich Place   

   9th Most Common Venue 10th Most Common Venue  
78  Cantonese Restaurant   Caribbean Restaurant

#### Cluster 5

In [154]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
11          Etobicoke               4           Coffee Shop   
12        Scarborough               4           Coffee Shop   
14          East York               4                  Café   
15   Downtown Toronto               4                  Café   
20   Downtown Toronto               4                  Café   
21               York               4    Italian Restaurant   
28         North York               4           Coffee Shop   
29          East York               4           Coffee Shop   
30   Downtown Toronto               4           Coffee Shop   
35          East York               4      Greek Restaurant   
40         North York               4        Clothing Store   
45         North York               4           Coffee Shop   
48   Downtown Toronto               4                  Café   
50         North York               4           Coffee Shop   
51        Scarborough               4       Harbor / Marina   
54       East Toronto               4           Coffee Shop   
55         North York               4           Coffee Shop   
57         North York               4           Coffee Shop   
63               York               4           Coffee Shop   
65        Scarborough               4           Coffee Shop   
68    Central Toronto               4           Coffee Shop   
70          Etobicoke               4           Coffee Shop   
74    Central Toronto               4           Coffee Shop   
76        Mississauga               4           Coffee Shop   
77          Etobicoke               4           Coffee Shop   
79    Central Toronto               4                  Café   
85        Scarborough               4    Chinese Restaurant   
86    Central Toronto               4    Italian Restaurant   
87   Downtown Toronto               4                  Park   
88          Etobicoke               4                  Park   
89          Etobicoke               4           Coffee Shop   
90        Scarborough               4    Chinese Restaurant   
91   Downtown Toronto               4    Italian Restaurant   
101         Etobicoke               4           Coffee Shop   

    2nd Most Common Venue   3rd Most Common Venue      4th Most Common Venue  \
11      Convenience Store             Pizza Place                       Bank   
12         Sandwich Place          Breakfast Spot                       Bank   
14                   Park             Coffee Shop                Pizza Place   
15            Coffee Shop                   Hotel                  Gastropub   
20     Italian Restaurant             Coffee Shop                  Gastropub   
21            Coffee Shop                  Bakery             Sandwich Place   
28               Pharmacy             Pizza Place             Sandwich Place   
29         Sandwich Place                    Park                     Bakery   
30                  Hotel                    Café                    Theater   
35                   Café                  Bakery                Coffee Shop   
40            Coffee Shop      Athletics & Sports        Sporting Goods Shop   
45                   Bank            Burger Joint                       Café   
48                  Hotel                 Theater                Coffee Shop   
50   Fast Food Restaurant                    Park                       Bank   
51         Sandwich Place             Coffee Shop                       Park   
54                 Bakery                    Park                       Café   
55                 Bakery        Sushi Restaurant             Sandwich Place   
57         Sandwich Place  Furniture / Home Store                 Beer Store   
63                 Bakery      Italian Restaurant                Pizza Place   
65   Fast Food Restaurant          Sandwich Place         Chinese Restaurant   
68     Italian Restaurant                    Café           Sushi Restaurant   
70         Sandwich Place             Pizza Plac